<img src='../../img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Datashader Exercises: LandSat

Creating Custom Transfer Functions for LandSat8

## Table of Contents
* [Datashader Exercises: LandSat](#Datashader-Exercises:-LandSat)
	* [Setup](#Setup)
	* [Load Landsat Data](#Load-Landsat-Data)
	* [Datashader Transfer Functions](#Datashader-Transfer-Functions)
	* [Example: Plot Just the Blue Band](#Example:-Plot-Just-the-Blue-Band)
* [Exercises](#Exercises)
	* [Rescaling Data](#Rescaling-Data)
	* [Creating an Image from RGB Components](#Creating-an-Image-from-RGB-Components)
	* [Plot True Color](#Plot-True-Color)
	* [Plot Color Infrared (Vegetation)](#Plot-Color-Infrared-%28Vegetation%29)
	* [Plot False Color (Urban)](#Plot-False-Color-%28Urban%29)
	* [Plot False Color 2](#Plot-False-Color-2)
	* [Plot Land vs. Water](#Plot-Land-vs.-Water)
	* [Plot Shortwave Infrared](#Plot-Shortwave-Infrared)


## Setup

The following imports will be needed to complete the exercises or provide for an improved notebook display:

In [ ]:
from os import path

import numpy as np
import rasterio as rio

from bokeh.models import Range1d
from bokeh.plotting import Figure
from bokeh.io import output_notebook, show

from datashader.bokeh_ext import InteractiveImage
import datashader as ds
import datashader.transfer_functions as tf

from bokeh.tile_providers import STAMEN_TONER

output_notebook()

## Load Landsat Data 

Bands	Wavelength
(micrometers)	Resolution
(meters)
- Band 1 - Coastal aerosol	0.43 - 0.45	30
- Band 2 - Blue	0.45 - 0.51	30
- Band 3 - Green	0.53 - 0.59	30
- Band 4 - Red	0.64 - 0.67	30
- Band 5 - Near Infrared (NIR)	0.85 - 0.88	30
- Band 6 - SWIR 1	1.57 - 1.65	30
- Band 7 - SWIR 2	2.11 - 2.29	30
- Band 8 - Panchromatic	0.50 - 0.68	15
- Band 9 - Cirrus	1.36 - 1.38	30
- Band 10 - Thermal Infrared (TIRS) 1	10.60 - 11.19	100 * (30)
- Band 11 - Thermal Infrared (TIRS) 2	11.50 - 12.51	100 * (30)

In [ ]:
data_dir = '../../data/Datashader/'

band1 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B1.TIF'))
band2 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B2.TIF'))
band3 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B3.TIF'))
band4 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B4.TIF'))
band5 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B5.TIF'))
band6 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B6.TIF'))
band7 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B7.TIF'))
band8 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B8.TIF'))
band9 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B9.TIF'))
band10 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B10.TIF'))
band11 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B11.TIF'))
band12 = rio.open(path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_BQA.TIF'))

# Notice the MERCATOR prefix which indicates the data was project to Mercator CRS

# find LC80210392016114LGN00_*.TIF | parallel rio warp {} --dst-crs EPSG:3857 MERCATOR_{}

xmin = band1.bounds.left
ymin = band1.bounds.bottom
xmax = band1.bounds.right
ymax = band1.bounds.top

## Datashader Transfer Functions

In [ ]:
def base_plot(tools='pan,wheel_zoom,reset',plot_width=900, plot_height=500, x_range=None, y_range=None, **plot_args):
    p = Figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        background_fill_color='black',
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.add_tile(STAMEN_TONER)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

## Example: Plot Just the Blue Band

In [ ]:
def update_image(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    blue_img = tf.shade(cvs.raster(band2),
                              cmap=['black','white'],
                              how='linear')
    return blue_img

In [ ]:
p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, update_image)

# Exercises

## Rescaling Data

Write a function which:

- takes a numpy array as input
- returns re-scaled array such that all values are between 0-255
- bonus: enhance contrast and brightness

In [ ]:
from xarray import DataArray
from datashader.utils import ngjit

In [ ]:
@ngjit
def normalize_data(agg):
    out = np.zeros_like(agg)
    min_val = 0
    max_val = 2**16 - 1
    range_val = max_val - min_val
    col, rows = agg.shape
    c = 25
    th = .125
    for x in range(col):
        for y in range(rows):
            val = agg[x, y]
            norm = (val - min_val) / range_val
            norm = 1 / (1 + np.exp(c * (th - norm))) # bonus
            out[x, y] = norm * 255.0
    return out

## Creating an Image from RGB Components

Write a function which:

- accept three numpy arrays named red, green, blue
- returns a `tf.Image` object

In [ ]:
def combine_bands(r, g, b):
    r = (normalize_data(r)).astype(np.uint8)
    g = (normalize_data(g)).astype(np.uint8)
    b = (normalize_data(b)).astype(np.uint8)
    col, rows = r.shape
    a = (np.zeros_like(r) + 255).astype(np.uint8)
    img = np.dstack([r, g, b, a]).view(np.uint32).reshape(r.shape)
    return tf.Image(data=img)

## Plot True Color 

(Red=Red, Green=Green, Blue=Blue)

In [ ]:
def true_color(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b).data for b in (band4, band3, band2)]
    return combine_bands(r, g, b)

In [ ]:
p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, true_color)

## Plot Color Infrared (Vegetation) 

(Red=Near Infrared, Green=Red, Blue=Green)

In [ ]:
def color_infrared(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b).data for b in (band5, band4, band3)]
    return combine_bands(r, g, b)

In [ ]:
p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, color_infrared)

## Plot False Color (Urban)

(Red=SWIR 2, Green=SWIR 1, Blue=Red)

In [ ]:
def false_color_urban(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b).data for b in (band7, band6, band4)]
    return combine_bands(r, g, b)

In [ ]:
p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, false_color_urban)

## Plot False Color 2

(Red=Near Infrared, Green=SWIR 1, Blue=Coastal)

In [ ]:
def false_color_veg(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b).data for b in (band5, band7, band1)]
    return combine_bands(r, g, b)

In [ ]:
p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, false_color_veg)

## Plot Land vs. Water

(Red=Near Infrared, Green=SWIR 1, Blue=Red)

In [ ]:
def land_vs_water(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b).data for b in (band5, band6, band4)]
    return combine_bands(r, g, b)

In [ ]:
p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, land_vs_water)

## Plot Shortwave Infrared

(Red=SWIR2, Green=Near Infrared, Blue=Red)

In [ ]:
def shortwave_infrared(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b).data for b in (band7, band5, band4)]
    return combine_bands(r, g, b)

In [ ]:
p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, shortwave_infrared)

---
*Copyright Continuum 2012-2016 All Rights Reserved.*